In [ ]:
%pip install unstructured libmagic python-magic langchain tiktoken faiss-cpu

In [ ]:
%pip install langchain-google-genai langsmith langserve langgraph python-dotenv

In [ ]:
%pip install langchain_community

Import Libraries and Environment variables

In [ ]:
import langchain
import time
import pickle
import os
from dotenv import load_dotenv
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.document_loaders import UnstructuredURLLoader
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain.llms import Ollama

load_dotenv()

In [ ]:
loader = UnstructuredURLLoader(urls=[
    "https://www.siemens.com/global/en/products/services/gbs/operations/digital-solutions.html",
    "https://www.siemens.com/global/en/products/services/gbs/opportunity-to-cash.html"
])
data = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20, length_function=len)
texts = text_splitter.split_documents(data)

In [ ]:
embeddings = OllamaEmbeddings()

In [ ]:
#docsearch = FAISS.from_documents(texts, embeddings)

In [53]:
#Storing vector index in a local directory
import pickle
file_path = "vector_index.pkl"
with open(file_path, 'wb') as f:
    pickle.dump(docsearch, f)

In [ ]:
import json
import pickle
if os.path.exists(file_path):
    with open(file_path, 'rb') as f:
        docsearch = pickle.load(f)

In [ ]:
llm = Ollama(model='llama2')

In [ ]:
#import chatprompttemplate 
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# Create the RAG prompt
rag_prompt = ChatPromptTemplate(
    input_variables=['context', 'question'], 
    messages=[
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['context', 'question'], 
                template="""You answer questions about the Global business services. 
                Use only the provided documents as context to answer the question. 
                Do not use any additional information.
                If you don't know the answer, just say that you don't know. Do not use external knowledge. 
                Use three sentences maximum and keep the answer concise. 
                Make sure to reference your sources with quotes of the provided context as citations.
                \nQuestion: {question} \nContext: {context} \nAnswer:"""
                )
        )
    ]
)

# load in qa_chain
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff", prompt=rag_prompt)

In [54]:
query = "What is opportunity to cash domain?"

# Find similar documents to the search query
docs = docsearch.similarity_search(query)
print(docs)

[Document(page_content='Read our stories', metadata={'source': 'https://www.siemens.com/global/en/products/services/gbs/operations/digital-solutions.html'}), Document(page_content='Read our stories', metadata={'source': 'https://www.siemens.com/global/en/products/services/gbs/operations/digital-solutions.html'}), Document(page_content='Read our stories', metadata={'source': 'https://www.siemens.com/global/en/products/services/gbs/opportunity-to-cash.html'}), Document(page_content='Read our stories', metadata={'source': 'https://www.siemens.com/global/en/products/services/gbs/opportunity-to-cash.html'})]


In [55]:
# Set a response variable to the output of the chain
response = chain({"input_documents": docs, "question": query}, return_only_outputs=True)

# Display the response
print(response["output_text"])

Based on the provided context, I can infer that "opportunity to cash" refers to the process of turning an idea or concept into a viable business opportunity. This domain is likely related to entrepreneurship and innovation, as it involves identifying potential opportunities and turning them into tangible products or services that can be monetized.

Here are three sentences that summarize my understanding of the term "opportunity to cash" based on the provided context:

1. The opportunity to cash refers to the process of transforming an idea or concept into a feasible business opportunity. (Source: Read our stories)
2. This domain involves identifying potential opportunities and developing them into tangible products or services that can be monetized. (Source: Read our stories)
3. The opportunity to cash is related to entrepreneurship and innovation, as it requires the ability to recognize potential opportunities and bring them to fruition. (Source: Read our stories)


In [ ]:
print(response)